# 전작업
- Colab GPU 설정
- 구글 드라이브 연동

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Yolo V4 설치

## Github에서 clone

```bash
git clone https://github.com/AlexeyAB/darknet
```
- yolo 공식 홈페이지: https://pjreddie.com/darknet/yolo/
- yolo 공식 github: https://github.com/pjreddie/darknet
   - yolo 공식 github에서는 Linux 운영체제만 지원한다. 
- AlexeyAB
   - https://github.com/AlexeyAB/darknet
   - yolo 오리지널을 fork 해서 리눅스와 윈도우를 모두 지원하도록 구현.
   - [Windows 설치](https://github.com/AlexeyAB/darknet#how-to-compile-on-windows-using-cmake)


In [ ]:
#yolov4 clone
!git clone https://github.com/AlexeyAB/darknet.git

## make 를 위해 구동환경 옵션 변경
- 다운받은 모델의 설치를 위해 make를 위한 구동환경 옵션을 변경
    - darknet/Makefile 파일의 내용을 변경한다.

> - **make**: 리눅스상에서 C 컴파일을 쉽게 해주는 프로그램    
> - **makefile**: make가 컴파일 하는 과정을 정의한 설정파일
```
GPU=0
CUDNN=0
CUDNN_HALF=0
OPENCV=0
```
을 1로 변경
- CPU 환경일 경우 OPENCV=1 만 변경한다.

## make를 이용해 Yolo V4 설치

In [ ]:
%cd darknet

In [ ]:
!make

## Pretrained Yolo V4 Weights 다운 로드

YOLO v4는 [coco dataset](https://cocodataset.org/#home)의 80 class를 학습한 pretrained weight를 제공한다. 이것을 다운받으면 80개 클래스에 속한 object들에 대한 detection은 추가 학습 없이 할 수 있다.

- `!wget -P 저장경로  https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights`

> url의 파일을 다운로드 받는 리눅스 명령어
> - wget -P <다운 받을 경로>  url

In [ ]:
!wget -P pretrained_weight https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

### Image 출력함수 정의

In [ ]:
import cv2
import matplotlib.pyplot as plt

def im_show(image_path):
    """
    매개변수로 이미지 경로를 받아서 그 이미지를 출력
    """
    image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(15,15))
    plt.imshow(image)
    plt.show()


# 기본 경로 변수 정의

In [ ]:
import os
ROOT_PATH = "/content/drive/MyDrive/object_detection/lion_tiger_detection_yolo"
# 테스트 이미지가 저장된 디렉토리 경로
TEST_DATA_PATH = os.path.join(ROOT_PATH, "test_data")
# 위에서 다운받은 Pre trained weight 저장 디렉토리 경로
PRED_TRAINED_WEIGTH_PATH = "/content/darknet/pretrained_weight/yolov4.weights"

In [ ]:
# 이미지 확인
im_show(os.path.join(TEST_DATA_PATH, 'hiway.jpg'))

# Yolo V4 Object Detection 실행
- Pretrained Weight를 이용해 영상안의 물체 검출
- teminal 환경에서 **darknet 명령어**를 이용해 실행

## darknet 명령어 옵션
`darknet detector test <path to .data file> <path to config> <path to weights> <path to image> <flags>`

- \<path to .data file\>
    - `.data` 파일경로. .data파일은 Train/Test dataset 파일경로목록, .names 파일(클래스들 설정)등의 경로를 지정한 파일.
- \<path to config\>
    - .config 파일경로: 모델구조, Train/Test 관련 설정파일
- \<path to weights\>
    - 데이터를 학습시킨 weight 파일의 경로
- \<path to image\>
    - 추론(Detection)할 image 파일경로
- \<flag\>
    - 실행 옵션
- detect가 끝나면 결과를 "predictions.jpg" 로 저장한다.    

## Image Detection

## 동영상 Detection
darknet detetor demo <.data>  <.cfg>  <weights>  검출대상동영상경로  -out_filename  결과영상저장경로  -dont_show

In [ ]:
# 테스트할 동영상 파일을 Google Drive에서 local로 복사
f"!cp {TEST_DATA_PATH}/test.mp4  /content/darknet/data"

In [ ]:
f"!./darknet  detector  demo  cfg/coco.data   cfg/yolov4.cfg  {PRED_TRAINED_WEIGTH_PATH}  \
data/test.mp4  -out_filename  test_result.avi  -dont_show"

## detection 실행 옵션

### 추론 결과 이미지가 안나오도록 설정.
- `-dont_show` flag
- colab 에서는 이미지/영상 출력이 안되기 때문에 이 flag를 명시해준다. 

In [ ]:
f"!./darknet detector test  cfg/coco.data  cfg/yolov4.cfg  {PRED_TRAINED_WEIGTH_PATH}  data/person.jpg  -dont_show"

In [ ]:
im_show("predictions.jpg")

### Threshold Flag
- `-thresh`
    - detection 결과의 confidence score의 threshold(임계값) 설정. 
    - ex) -thresh 0.7 : 0.7이상의 confidence score인 것만 detection 결과로 나온다.

In [ ]:
f"!./darknet detector test  cfg/coco.data  cfg/yolov4.cfg  /content/darknet/pretrained_weight/yolov4.weights  {TEST_DATA_PATH}/hiway.jpg -dont_show"

In [ ]:
im_show("predictions.jpg")

In [ ]:
# -thresh 옵션 설정
# ex) -thresh 0.9 : Confidence score가 0.9이상인 것만 bbox를 출력
f"!./darknet detector test  cfg/coco.data  cfg/yolov4.cfg  /content/darknet/pretrained_weight/yolov4.weights  {TEST_DATA_PATH}/hiway.jpg -dont_show -thresh  0.9"

In [ ]:
im_show('predictions.jpg')

### 출력결과에 Bounding Box 좌표(Coordinate) 출력
- `-ext_output`

In [ ]:
f"!./darknet detector test  cfg/coco.data  cfg/yolov4.cfg  /content/darknet/pretrained_weight/yolov4.weights  {TEST_DATA_PATH}/hiway.jpg -dont_show -thresh  0.9 -ext_output"

In [ ]:
im_show('predictions.jpg')

### 결과 파일 저장
- `-out 파일명`
    - 파일명의 확장자를 `.json`으로 지정하면 json format으로 저장.

In [ ]:
f"!./darknet detector test  cfg/coco.data  cfg/yolov4.cfg  /content/darknet/pretrained_weight/yolov4.weights  {TEST_DATA_PATH}/hiway.jpg -dont_show -thresh  0.9   -out  results/hiway_result.json"

### 한번에 여러 이미지 Detection
- text 파일에 Detection할 이미지 경로 목록을 작성한다. 
    - 한줄에 한 파일씩 경로를 작성한다. (절대경로로 작성한다)
    - darknet 명령문 뒤에 `< 파일경로` 를 추가

```
darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights -ext_output -thresh 0.5 < data/images.txt
```

In [ ]:
f_list = [
        "data/person.jpg\n", 
        "data/eagle.jpg\n",
        "data/dog.jpg\n"
]
with open('data/image_list.txt', 'wt') as f:
    f.writelines(f_list)

In [ ]:
f"!./darknet detector test  cfg/coco.data  cfg/yolov4.cfg  /content/darknet/pretrained_weight/yolov4.weights  {TEST_DATA_PATH}/hiway.jpg -dont_show -thresh  0.5   -out  results/hiway_result.json < data/image_list.txt"

# Custom Dataset 학습

## 준비
1. Custom Dataset 준비
    - Image dataset과 Labeling한 annotation 파일 준비
1. .data, .names 파일 준비
1. TRAIN/VAL/TEST 이미지 목록파일.
1. .cfg (config) 파일 준비
1. convolution layer를 위한 Pretrained 모델 다운로드

## 학습할 Custom dataset 이미지 준비와 Labeling
1. open image dataset (ms coco, pascal voc, open images dataset)을 이용해 image와 annotation 수집
    - open dataset의 경우 image와 annotation 파일을 제공하므로 Labelling 작업이 용이하다. 그러나 open dataset 들마다 annotation 방식이 다르기 때문에 yolo annotation 방식에 맞게 변환하는 작업이 필요하다.
2. 크롤링등을 이용해 직접 수집 후 labeling 작업
    - Labeling Tool 을 이용해 수집한 이미지에 bounding box 작업을 한다.
    
### Yolo Label 형식
- 이미지 데이터파일 별로 하나씩 작성하며 `.txt`로 저장한다.
- 한줄에 하나의 object에 대한 bounding box 정보를 작성한다.
- 형식
    - `<label> <bbox center x좌표> <bbox center y좌표> <bbox width> <bbox height>`
    - `0 0.331905 0.378493 0.568571 0.753599`
    - 공백을 구분자로 사용한다.
    - 좌표들은 이미지 width, height에 대한 비율로 작성한다.        

## 경로 변수 정의

In [ ]:
import os
# 프로젝트 root path
BASE_PATH = '/content/drive/MyDrive/object_detection/lion_tiger_detection_yolo'
# workspace 경로
WORKSPACE_PATH = os.path.join(BASE_PATH, 'workspace')
# scripts 디렉토리 경로
SCRIPT_PATH = os.path.join(BASE_PATH, 'scripts')

# Dataset 관련 경로
# image/annotation 저장 디렉토리 경로
DATASET_ROOT_PATH = '/content/images'
# Train/Validation/Test 이미지, annotation파일들의 경로
TRAIN_DATASET_PATH = os.path.join(DATASET_ROOT_PATH, 'train')
VAL_DATASET_PATH = os.path.join(DATASET_ROOT_PATH, 'val')
TEST_DATASET_PATH = os.path.join(DATASET_ROOT_PATH, 'test')

# 설정파일 관련 경로
# 설정파일들의 ROOT 경로
CONFIG_PATH = os.path.join(WORKSPACE_PATH, 'config')

# .names 파일의 경로 -> 분류할 문체들의 class를 설정한 파일(Label map 의 역할)
NAMES_FILE_PATH = os.path.join(CONFIG_PATH, 'obj.names')
# .data 파일의 경로 -> 학습할 데이터, 학습관련사항을 설정한 파일
DATA_FILE_PATH = os.path.join(CONFIG_PATH, 'obj.data')
# cfg(config) 파일의 경로 -> yolo v4 모델의 구조관련 설정
MODEL_CONFIG_FILE_PATH = os.path.join(CONFIG_PATH, "yolov4.cfg")

# train/validation/test 이미지 파일 목록 설정 파일의 경로 - 모든 이미지 파일들의 경로를 작성한 목록파일
TRAIN_LIST_FILE_PATH = os.path.join(CONFIG_PATH, 'train_list.txt') 
VAL_LIST_FILE_PATH = os.path.join(CONFIG_PATH, 'val_list.txt')
TEST_LIST_FILE_PATH = os.path.join(CONFIG_PATH, 'test_list.txt')

# weights 관련 path
# custom data 학습 과정에서 생기는 weight 파일들을 저장(backup)할 경로
WEIGHT_BACKUP_PATH = os.path.join(WORKSPACE_PATH, 'weight_backup')
# 다운받은 pretrained weight를 저장할 경로
PRE_TRAINED_WEIGHT_DIR_PATH = os.path.join(WORKSPACE_PATH, 'pretrained_weight')
PRE_TRAINED_WEIGHT_FILE_PATH = os.path.join(PRE_TRAINED_WEIGHT_DIR_PATH, 'yolov4.conv.137')


## Dataset 압축 풀기
- 구글 드라이브에 저장된 lion_tiger.zip 파일을 local directory /content/images에 압축 풀기

In [ ]:
f"!unzip -q {os.path.join(BASE_PATH, 'raw_data', 'lion_tiger.zip')} -d {DATASET_ROOT_PATH}"

## Train을 위한 설정파일 작성

### dataset 경로 목록 파일
- 학습/검증/평가할 때 사용할 이미지들의 경로를 작성한 목록 파일을 작성한다.
    - train/validation/test 데이터셋 별로 작성한다. 
        - 예를 들어 train.txt 파일에는 train dataset의 모든 이미지의 경로를 작성한다.
    - 한줄에 한개의 파일씩 경로를 작성한다.
    - 경로는 절대경로, 상대경로 관계없다. 상대경로의 경우 학습을 실행한 디렉토리 기준으로 지정한다.

In [ ]:
# train set  파일 목록
f"!python {os.path.join(SCRIPT_PATH, 'make_file_list.py')} \
--directory {TRAIN_DATASET_PATH} --output {TRAIN_LIST_FILE_PATH}"

In [ ]:
# VALIDATION SET 파일 목록
f"!python {os.path.join(SCRIPT_PATH, 'make_file_list.py')} --directory {VAL_DATASET_PATH} --output {VAL_LIST_FILE_PATH}"

In [ ]:
# Test set 파일 목록
f"!python {os.path.join(SCRIPT_PATH, 'make_file_list.py')} --directory {TEST_DATASET_PATH} --output {TEST_LIST_FILE_PATH}"

### <파일명>.names 파일 작성
- .names 파일
    - ex) obj.names
    - detection할 물체(object)들의 class 들을 작성한 파일
    - class 이름을 한줄에 하나씩 작성한다.
    
```
lion
tiger
```

In [ ]:
label_list = ['lion\n', 'tiger']
with open(NAMES_FILE_PATH, 'wt') as fw:
    fw.writelines(label_list)

### <파일명>.data 파일 작성

- .data 파일
    - 학습할 데이터셋과 과련된 설정을 하는 파일.
    - ex) obj.data
```
classes = 2
train = data/train.txt
valid = data/validation.txt
names = data/obj.names
backup = backup/
```
    - classes: 검출(detection)할 물체(object)의 개수
    - train: Train dataset의 이미지들 경로 목록 파일. 
    - vaild: Validation dataset의 이미지들 경로 목록파일.
    - names: .names 파일 경로.
    - backup: 학습 중간 결과 weight들을 저장할 디렉토리.

In [ ]:
data_dict = {
    'classes':2,
    'train':TRAIN_LIST_FILE_PATH,
    'valid':VAL_LIST_FILE_PATH,
    'names':NAMES_FILE_PATH,
    'backup':WEIGHT_BACKUP_PATH
}

with open(DATA_FILE_PATH, 'wt') as fw:
    for key, value in data_dict.items():
        fw.write(f"{key}={value}\n")

### .cfg (config) 파일 준비
- 학습에 사용할 모델관련 설정파일
- cfg/yolov4.cfg 파일을 복사한 뒤 다음 항목들을 수정한다.
    - batch
    - subdivisions
        - mini-batch의 darknet 용어.
        - GPU 메모리가 부족할 경우 subdivisions의 값을 낮게 잡아준다.
    - max_batches(19줄)
        - 반복횟수 iteration으로 위에서 지정한 batch를 몇번 반복할지 지정.
        - 추천설정: class수 * 2000 (2000 ~ 4000) + class개수*100
            - class:2 -> 2\*2000 + 2\*100 : 4200
    - steps (21줄)
        - 추천설정: (80% of max_batches), (90% of max_batches)
    - \[yolo\] 검색 (3군데)
        - \[yolo\] 바로위의 \[convolutional\] 설정의 filters를 (class수+5)*3 으로 변경
            - 961, 1049, 1137 라인
        - \[yolo\] 설정중 classes: 클래스 개수 로 변경
            - 968, 1056, 1144 라인

In [ ]:
# cfg/yolov4.cfg -복사-> workspace/config
f"!cp /content/darknet/cfg/yolov4.cfg   {MODEL_CONFIG_FILE_PATH}"

##  Convolution layer를 위한 Pretrained weight 다운로드
- 전체 모델중 Yolo V4의 backbone network(Feature extractor)의 미리 학습 시킨 가중치(Weight)를 다운받아 Train때 사용한다. 
- Download url
    -  https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

In [ ]:
f"!wget -P {PRE_TRAINED_WEIGHT_DIR_PATH}  https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137"

## Custom Data 학습하기

- 명령어
```
!./darknet detector train <path to .data> <path to custom config> <path to weights> -dont_show -map
```
    - \<path to .data file\>: `.data` 파일경로. 
    - \<path to custom config\>: .config 파일경로
    - \<path to weights\>: pretrained weight 파일 경로
    - `-dont_show`
        - 학습 진행 chart를 pop으로 보여주는데 Colab에서는 이 chart를 볼 수 없기 때문에 보여주지 말도록 설정. 대신 학습 후 chart.jpg 파일이 생성된다.
    - `-map` : Mean Average Precision을 평가 지표로 사용. mAP로 평가결과가 학습 진행 chart에 출력된다.
 
- 학습 100번의 iteration 마다 `.data` 에 설장한 backup 경로에 yolov4_last.weight 파일로 학습된 weights를 저장한다. 
    - 혹시 중간에 문제가 생겨 학습이 멈추면 이 가중치를 이용해 학습을 이어나가면 된다.
- 학습이 완료되면 .data 설정파일에 지정한 backup 디렉토리에 yolov4_best.weights, yolov4_last.weights, yolov4_final.weights, yolov4_1000.weights, yolov4_2000.weights,... 파일이 생성된다.

In [ ]:
f"!./darknet  detector  train  {DATA_FILE_PATH}  {MODEL_CONFIG_FILE_PATH}  {PRE_TRAINED_WEIGHT_FILE_PATH} \
 -dont_show  -map"

In [ ]:
!pip install --upgrade gdown

In [ ]:
# 미리 학습 시킨 weight 다운로드

import gdown
url = 'https://drive.google.com/uc?id=1yWVk-Tavo644s_byVrvk9Ju7U_imQwwZ'
fname = 'yolov4_best.weights'
gdown.download(url, fname, quiet=False)

In [ ]:
f"!cp yolov4_best.weights  -d {WEIGHT_BACKUP_PATH}"

# 새로운 데이터 추론

In [ ]:
file_path = 'data/tiger2.jpg'
f"!./darknet  detector  test  {DATA_FILE_PATH}  {MODEL_CONFIG_FILE_PATH}  yolov4_best.weights \
 {file_path}  -dont_show"

In [ ]:
im_show('predictions.jpg')